In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import re

In [ ]:
!git clone https://github.com/tatnashev/prozhito

fatal: destination path 'prozhito' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/hse-cl-masterskaya-prozhito/main

Cloning into 'main'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 24 (delta 4), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [ ]:
df_train = pd.read_csv('/content/main/data/train_df_new.csv') 
df_test = pd.read_csv('/content/main/data/test_df_new.csv') 
df_val = pd.read_csv('/content/main/data/val_df_new.csv') 

df = pd.concat([df_train, df_test, df_val]) 
df.shape[0]

1678

## Размечено  

In [ ]:
label_list = ['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-FAC', 'I-FAC', 'B-CHAR', 'I-CHAR', 'O']
num2bio = dict(enumerate(label_list))

In [ ]:
def preprocess_labels(labels): 
    nums = labels.split(', ')
    nums[0] = nums[0][1:] 
    nums[-1] = nums[-1][:-1]

    return list(map(int, nums))

In [ ]:
def get_bio(bio_nums): 
    return [num2bio[elem] for elem in bio_nums]

In [ ]:
df_train = pd.read_csv('prozhito/prozhito_data/df_train_prozhito.csv') 
df_test = pd.read_csv('prozhito/prozhito_data/df_test_prozhito.csv') 
df_val = pd.read_csv('prozhito/prozhito_data/df_val_prozhito.csv') 

df_prozhito = pd.concat([df_train, df_test, df_val]) 
df_prozhito.head()

,Unnamed: 0,tokens,BIO_str,BIO_nums,BIO_list
0,0,У меня большая симпатия к Лукьянину — человек ...,O O O O O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,2,"> Каким приговором , указом каким > Ты здесь ,...",O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,3,"Подумал , что летом ребята куда затащили .",O O O O B-CHAR O O O,"[10, 10, 10, 10, 8, 10, 10, 10]","['O', 'O', 'O', 'O', 'B-CHAR', 'O', 'O', 'O']"
3,4,Нашел потрясающие материалы о В . М . Брадисе ...,O O O O B-PER I-PER I-PER I-PER I-PER O O B-CH...,"[10, 10, 10, 10, 4, 5, 5, 5, 5, 10, 10, 8, 8, ...","['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'..."
4,5,[Без даты . ],O O O O,"[10, 10, 10, 10]","['O', 'O', 'O', 'O']"


In [ ]:
texts = [] 
texts.extend(df_train['tokens'].values) 
texts.extend(df_test['tokens'].values) 
texts.extend(df_val['tokens'].values) 

len(list(set(texts)))

1364

In [ ]:
df_prozhito = df_prozhito.drop(columns=['Unnamed: 0'])
df_prozhito['BIO_nums'] = df_prozhito['BIO_nums'].apply(lambda x: preprocess_labels(x)) 
df_prozhito['BIO'] = df_prozhito['BIO_nums'].apply(lambda x: get_bio(x)) 
df_prozhito = df_prozhito.drop(columns=['BIO_str', 'BIO_list'])
df_prozhito.head() 

,tokens,BIO_nums,BIO
0,У меня большая симпатия к Лукьянину — человек ...,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"> Каким приговором , указом каким > Ты здесь ,...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"Подумал , что летом ребята куда затащили .","[10, 10, 10, 10, 8, 10, 10, 10]","[O, O, O, O, B-CHAR, O, O, O]"
3,Нашел потрясающие материалы о В . М . Брадисе ...,"[10, 10, 10, 10, 4, 5, 5, 5, 5, 10, 10, 8, 8, ...","[O, O, O, O, B-PER, I-PER, I-PER, I-PER, I-PER..."
4,[Без даты . ],"[10, 10, 10, 10]","[O, O, O, O]"


In [ ]:
print(f'Размер Размечено: {df_prozhito.shape[0]}')

Размер Размечено: 1678


## FactRu

In [ ]:
def preprocess_bio(bio): 
    regex = re.compile('[A-Z-]+')
    return regex.findall(bio)

In [ ]:
df_train = pd.read_csv('prozhito/data/factru_train.csv') 
df_test = pd.read_csv('prozhito/data/factru_test.csv') 
df_val = pd.read_csv('prozhito/data/factru_val.csv') 

df_factru = pd.concat([df_train, df_test, df_val])  
df_factru.head()

,tokens,BIO
0,"['Я', 'голосовал', 'против', 'увольнения', 'Фе...","['O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', '..."
1,"['Белоруссия', ',', 'Казахстан', ',', 'как', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['В', 'то', 'же', 'время', 'канцлер', 'подчерк...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Спуск', 'корабля', 'с', 'орбиты', 'проходил'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Герой', 'Мураками', 'увлекается', 'западными...","['O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', '..."


In [ ]:
df_factru['BIO'] = df_factru['BIO'].apply(lambda x: preprocess_bio(x)) 
df_factru.head()

,tokens,BIO
0,"['Я', 'голосовал', 'против', 'увольнения', 'Фе...","[O, O, O, O, B-PER, O, O, O, O, O, B-PER, O, O..."
1,"['Белоруссия', ',', 'Казахстан', ',', 'как', '...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"['В', 'то', 'же', 'время', 'канцлер', 'подчерк...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,"['Спуск', 'корабля', 'с', 'орбиты', 'проходил'...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"['Герой', 'Мураками', 'увлекается', 'западными...","[O, B-PER, O, O, O, O, O, O, O, O]"


In [ ]:
print(f'Размер Factru: {len(df_factru)}')

Размер Factru: 4515


In [ ]:
texts = [] 
texts.extend(df_train['tokens'].values) 
texts.extend(df_test['tokens'].values) 
texts.extend(df_val['tokens'].values) 

len(list(set(texts)))

4454

## ne5 

In [ ]:
df_train = pd.read_csv('prozhito/data/ne5_train.csv')  
df_test = pd.read_csv('prozhito/data/ne5_test.csv')
df_val = pd.read_csv('prozhito/data/ne5_val.csv')

df_ne5 = pd.concat([df_train, df_test, df_val]) 
df_ne5.head()

,tokens,BIO
0,"['По', 'данным', 'министерства', 'здравоохране...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Шульц', 'намерен', 'содействовать', 'более',...","['I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,"['В', 'настоящее', 'время', 'инициированы', 'п...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Судя', 'по', 'всему', ',', 'непосредственно'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Голосование', 'проходило', 'в', 'открытой', ...","['O', 'O', 'O', 'O', 'O', 'O']"


In [ ]:
len(df_ne5)

15138

In [ ]:
df_ne5['BIO'] = df_ne5['BIO'].apply(lambda x: preprocess_bio(x)) 
df_ne5

,tokens,BIO
0,"['По', 'данным', 'министерства', 'здравоохране...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"['Шульц', 'намерен', 'содействовать', 'более',...","[I-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2,"['В', 'настоящее', 'время', 'инициированы', 'п...","[O, O, O, O, O, O, O, O, O, O, O]"
3,"['Судя', 'по', 'всему', ',', 'непосредственно'...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"['Голосование', 'проходило', 'в', 'открытой', ...","[O, O, O, O, O, O]"
...,...,...
1494,"['""', 'Надеюсь', ',', 'с', 'его', 'приходом', ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1495,"['Комиссия', 'по', 'расследованию', 'вторжения...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1496,"['Кузнецов', 'на', 'протяжении', 'пяти', 'лет'...","[I-PER, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG,..."
1497,"['По', 'данным', 'The', 'New', 'York', 'Times'...","[O, O, O, O, O, O, O, O, O, O, O, B-PER, O, O,..."


## bsnlp

In [ ]:
df_train = pd.read_csv('/content/prozhito/data/bsnlp_train (2).csv')  
df_test = pd.read_csv('/content/prozhito/data/bsnlp_test.csv')
df_val = pd.read_csv('/content/prozhito/data/bsnlp_val (2).csv')

df_bsnlp = pd.concat([df_train, df_test, df_val]) 
df_bsnlp.head()

,tokens,BIO
0,"['В', 'ЕС', 'ранее', 'отмечали', ',', 'что', '...","['O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', '..."
1,"['До', 'нового', 'назначения', 'Рааб', 'занима...","['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'B-ORG..."
2,"['Это', 'противоречит', 'позиции', '""', 'еарос...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Асия', 'Биби', 'был', 'приговорена', 'к', 'с...","['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O..."
4,"['<', '…', '>', 'Пакистан', '—', 'это', 'наша'...","['O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', '..."


In [ ]:
df_bsnlp['BIO'].values[0]

"['O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O']"

In [ ]:
df_bsnlp['BIO'] = df_bsnlp['BIO'].apply(lambda x: preprocess_bio(x))
df_bsnlp.head()

,tokens,BIO
0,"['В', 'ЕС', 'ранее', 'отмечали', ',', 'что', '...","[O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O,..."
1,"['До', 'нового', 'назначения', 'Рааб', 'занима...","[O, O, O, B-PER, O, O, O, B-ORG, I-ORG, I-ORG, O]"
2,"['Это', 'противоречит', 'позиции', '""', 'еарос...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"['Асия', 'Биби', 'был', 'приговорена', 'к', 'с...","[B-PER, I-PER, O, O, O, O, O, O, O, O, O]"
4,"['<', '…', '>', 'Пакистан', '—', 'это', 'наша'...","[O, O, O, B-LOC, O, O, O, O, O, O, B-PER, O, O..."


In [ ]:
len(df_bsnlp)

5614

## Всякие статистики

In [ ]:
def num_of_tokens(df): 
    regex = re.compile('[a-zA-ZА-Яа-я0-9.,!?...*^%#_=-]+')

    cnt_tokens = 0 
    for text in df['tokens'].values: 
        text = regex.findall(text) 
        cnt_tokens += len(text)

    return cnt_tokens

In [ ]:
def all_entities(df): 
    entities_set = set() 
    for bio in df['BIO'].values: 
        for tag in bio: 
            entities_set.add(tag.split('-')[-1]) 

    return list(entities_set) 

### Все количества 

In [ ]:
# количество уникальных токенов 

print(f'Уникальных токенов в FactRu: {num_of_tokens(df_factru)}') 
print(f'Уникальных токенов в bsnlp: {num_of_tokens(df_bsnlp)}') 
print(f'Уникальных токенов в ne5: {num_of_tokens(df_ne5)}')  
print(f'Уникальных токенов в размечено: {num_of_tokens(df_prozhito)}')  

Уникальных токенов в FactRu: 170341
Уникальных токенов в bsnlp: 205037
Уникальных токенов в ne5: 504141
Уникальных токенов в размечено: 21825


In [ ]:
# все сущности 

print(f'Сущности в FactRu: {all_entities(df_factru)}') 
print(f'Сущности в bsnlp: {all_entities(df_bsnlp)}') 
print(f'Сущности в ne5: {all_entities(df_ne5)}') 

### Плотность сущностей 

In [ ]:
def entities_density(BIO):
    cnt_all_tokens = 0 
    cnt_entities = 0 

    regex = re.compile('[A-Z]-[A-Z]+') 
    for bio in BIO: 
        cnt_all_tokens += len(bio) 

        joined_bio = ', '.join(bio) 
        cnt_entities += len(regex.findall(joined_bio))

    return cnt_entities / cnt_all_tokens

In [ ]:
bio = df_prozhito['BIO'].values[3]
bio

['O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'I-PER',
 'I-PER',
 'I-PER',
 'O',
 'O',
 'B-CHAR',
 'B-CHAR',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
density_prozhito = round(entities_density(df_prozhito['BIO'].values), 3)
print(f'Плотность сущностей в Размечено: {density_prozhito}')

Плотность сущностей в Размечено: 0.05


In [ ]:
density_factru = round(entities_density(df_factru['BIO'].values), 3)
print(f'Плотность сущностей в FactRu: {density_factru}')

Плотность сущностей в FactRu: 0.101


### Пересечение по топ-5000 самым частым словам (lower case и лемматизированные)

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 8.2 MB 39.3 MB/s 


In [ ]:
import pymorphy2 
morph = pymorphy2.MorphAnalyzer() 

In [ ]:
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist 
nltk.download('stopwords')

stopwords_set = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def find_all_words(texts):
    regex = re.compile('\w+')

    texts_only_words = []  
    for text in texts: 
        texts_only_words.append(regex.findall(text.lower())) 

    return texts_only_words 


def lemmatize_freq(texts, lemmer=morph, stopwords=stopwords_set):
    freq = FreqDist() 
    texts = find_all_words(texts) 

    new_texts = [] 
    for text in texts: 
        lemmas = [lemmer.parse(w)[0].normal_form for w in text if w.isalpha()]
        new_text = [] 

        for word in lemmas: 
            if word not in stopwords: 
                new_text.append(word) 
                freq[word] += 1

        new_texts.append(new_text) 

    return new_texts, freq

In [ ]:
prozhito_texts = df_prozhito['tokens'].values 
prozhito_lemmatized_texts, freq_prozhito = lemmatize_freq(prozhito_texts)

In [ ]:
factru_texts = df_factru['tokens'].values 
factru_lemmatized_texts, freq_factru = lemmatize_freq(factru_texts)

In [ ]:
prozhito_freq_5000 = set(dict(freq_prozhito.most_common(4000)).keys()) 
factru_freq_5000 = set(dict(freq_factru.most_common(4000)).keys()) 

In [ ]:
len(prozhito_freq_5000.intersection(factru_freq_5000))

1483

In [ ]:
print(f'Процент пересечения топ-4000 по встречаемости слов в датасетах: {(len(prozhito_freq_5000.intersection(factru_freq_5000)) / 4000) * 100}%')

Процент пересечения топ-4000 по встречаемости слов в датасетах: 37.075%
